In [11]:
from src.parsers import mimic, hirid
from src.modeling import discovery, plots, querier
from src.utils import constants

In [21]:
import sys
# import win32com.client
import os
import pandas as pd

def setup_io_config(root_path):
    """
    Input - Output config. Add dataset paths
    :root_path -> Repo path which contains 'data' and 'res' folders
    """

    # MIMIC
    is_shortcut = True if "data.lnk" in os.listdir(root_path) else False 
    
    if (is_shortcut):
        path_shortcut =  os.path.join(root_path, "data.lnk")
        shell = win32com.client.Dispatch("WScript.Shell")
        mimic_data = shell.CreateShortCut(path_shortcut).Targetpath
    else:
        mimic_data = os.path.join(f"{root_path}", "data") 
    mimic_path = os.path.join(f"{root_path}", "results")

    # HIRID
    hirid_data = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1/raw_stage/'
    hirid_path = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1'
    
    return mimic_data, mimic_path, hirid_data, hirid_path

def setup_stratification_config():
    gender="MF"
    age_b=40
    age_a=80 
    ethnicity="WHITE" 
    lab_mapping= constants.LAB_MAPPING
    before_windows = [(0,12)]
    after_windows = [(0,12)]
    return gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows

In [22]:
# IO Config
# root_path ="C:\\Users\\danco\\My Drive\\Master\\Datasets\\MIMIC iii"
root_path = "/Users/pavan/Library/CloudStorage/GoogleDrive-f20190038@hyderabad.bits-pilani.ac.in/My Drive/TAU/Code/DrugLab"
data, res, raw_path, res_path = setup_io_config(root_path=root_path)

# Stratification Config
gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows = setup_stratification_config()

In [4]:
import traceback
try:
    mimic_parser = mimic.MIMICParser(data=data, res=res, gender=gender, age_b=age_b, age_a=age_a, ethnicity=ethnicity, load="AUTOMATIC_MAPPING_MIMIC")
    m_med1, m_med2, m_labs = mimic_parser.parse(use_pairs=False, load_from_raw=False, load_raw_chartevents=False)
except:
    traceback.print_exc()

Loading med data...
Loaded med data.
Load 1st and 2nd medication data...
Loaded 1st and 2nd medication data.
Load Lab data...


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:225: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  labs = pd.read_csv(os.path.join(self.data, constants.MIMIC_III_PREPROCESSED_PATH, constants.MIMIC_III_PREPROCESSED_LABDATA))


Loaded Lab data.


In [23]:
med_lab_pairs = pd.read_csv("temp_med_lab_pairs.csv")

In [24]:
med_lab_pairs

,Unnamed: 0,SUBJECT_ID,OldITEMID,HADM_ID,ROW_ID_x,ROW_ID_x.1,ICUSTAY_ID,STARTTIME,ENDTIME,AMOUNT,...,CONCEPTID,AGE,MED_NAME,hours_in,LAB_ITEMID,"before_abs_(0, 12)_sp","before_time_(0, 12)_sp","after_abs_(0, 12)_sp","after_time_(0, 12)_sp",LAB_NAME
0,0,63024,225975,144024,3345728,36880,296980.0,2188-06-19 17:31:00,2188-06-19 17:32:00,1.000000,...,NaN,46.0,Heparin Sodium (Prophylaxis),-21.283333,Calcium,8.4,5.683333,NaN,NaN,Calcium
1,1,63024,225975,144024,3345728,36880,296980.0,2188-06-19 17:31:00,2188-06-19 17:32:00,1.000000,...,NaN,46.0,Heparin Sodium (Prophylaxis),-21.283333,Creatinine,0.7,5.683333,NaN,NaN,Creatinine
2,2,63024,225975,144024,3345728,36880,296980.0,2188-06-19 17:31:00,2188-06-19 17:32:00,1.000000,...,NaN,46.0,Heparin Sodium (Prophylaxis),-21.283333,Glucose,88.0,5.683333,NaN,NaN,Glucose
3,3,63024,225975,144024,3345728,36880,296980.0,2188-06-19 17:31:00,2188-06-19 17:32:00,1.000000,...,NaN,46.0,Heparin Sodium (Prophylaxis),-21.283333,Hematocrit,40.2,5.683333,NaN,NaN,Hematocrit
4,4,63024,225975,144024,3345728,36880,296980.0,2188-06-19 17:31:00,2188-06-19 17:32:00,1.000000,...,NaN,46.0,Heparin Sodium (Prophylaxis),-21.283333,Hemoglobin,12.4,5.683333,NaN,NaN,Hemoglobin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1827794,1827794,93321,225930,115396,3599677,44773,248745.0,2128-09-14 15:33:00,2128-09-14 18:43:00,31.666667,...,NaN,62.0,Peptamen 1.5 (Full),4171.700000,PTT,NaN,NaN,34.90,9.033333,PTT
1827795,1827795,93321,225930,115396,3599677,44773,248745.0,2128-09-14 15:33:00,2128-09-14 18:43:00,31.666667,...,NaN,62.0,Peptamen 1.5 (Full),4171.700000,Potassium,NaN,NaN,3.70,9.033333,Potassium
1827796,1827796,93321,225930,115396,3599677,44773,248745.0,2128-09-14 15:33:00,2128-09-14 18:43:00,31.666667,...,NaN,62.0,Peptamen 1.5 (Full),4171.700000,Prothrombin time INR,NaN,NaN,1.00,9.033333,Prothrombin time INR
1827797,1827797,93321,225930,115396,3599677,44773,248745.0,2128-09-14 15:33:00,2128-09-14 18:43:00,31.666667,...,NaN,62.0,Peptamen 1.5 (Full),4171.700000,Red blood cell,NaN,NaN,2.53,9.033333,Red blood cell


In [16]:
res_pvals = pd.read_csv("temp_pvals_med_lab_pairs_1-12.csv")

In [17]:
res_pvals

,Unnamed: 0,Lab Name,Med Name,Before Window (in Hours),After Window (in Hours),Mannwhitneyu Test,TTest Independent,TTest Paired
0,0,Alanine aminotransferase,ACD-A Citrate (1000ml),"(0, 12)","(0, 12)",0.771503,0.791319,0.386098
1,1,Albumin,ACD-A Citrate (1000ml),"(0, 12)","(0, 12)",0.121183,0.156159,0.129612
2,2,Amylase,ACD-A Citrate (1000ml),"(0, 12)","(0, 12)",1.000000,NaN,NaN
3,3,Asparate aminotransferase,ACD-A Citrate (1000ml),"(0, 12)","(0, 12)",0.885714,0.769740,0.366921
4,4,Bilirubin,ACD-A Citrate (1000ml),"(0, 12)","(0, 12)",1.000000,0.927804,0.489458
...,...,...,...,...,...,...,...,...
4154,4154,Potassium,Voriconazole,"(0, 12)","(0, 12)",0.553383,0.361353,0.082985
4155,4155,Prothrombin time INR,Voriconazole,"(0, 12)","(0, 12)",1.000000,0.869515,0.407659
4156,4156,Red blood cell,Voriconazole,"(0, 12)","(0, 12)",0.154390,0.122364,0.088458
4157,4157,Sodium,Voriconazole,"(0, 12)","(0, 12)",0.655973,0.829398,0.532569


In [18]:
analyzer = discovery.ClinicalDiscoveryAnalysis(med_lab_pairs)

In [ ]:
res_pvals = analyzer.analyze(before_windows=before_windows, after_windows=after_windows)

In [19]:
sig_pairs = analyzer.generate_significant(res_pvals)

In [20]:
sig_pairs[-1].sort_values("TTest Paired").head(100)

,Unnamed: 0,Lab Name,Med Name,Before Window (in Hours),After Window (in Hours),Mannwhitneyu Test,TTest Independent,TTest Paired,BonferroniPvals,FDR Benjamini Corrected
3734,3734,Magnesium,Replete with Fiber (1/2),"(0, 12)","(0, 12)",0.666667,0.683772,0.000000e+00,0.000000e+00,NaN
2367,2367,Alanine aminotransferase,Mannitol,"(0, 12)","(0, 12)",0.666667,0.980523,0.000000e+00,0.000000e+00,NaN
152,152,Lactic acid,Ambisome,"(0, 12)","(0, 12)",0.666667,0.552786,0.000000e+00,0.000000e+00,NaN
3891,3891,Lipase,TPN w/ Lipids,"(0, 12)","(0, 12)",0.666667,0.777175,0.000000e+00,0.000000e+00,NaN
2948,2948,Sodium,Nutren Renal (1/2),"(0, 12)","(0, 12)",0.666667,0.835601,0.000000e+00,0.000000e+00,NaN
...,...,...,...,...,...,...,...,...,...,...
1690,1690,Lactic acid,Heparin Sodium (Prophylaxis),"(0, 12)","(0, 12)",0.000163,0.000802,1.831732e-14,7.618173e-11,NaN
3844,3844,Hemoglobin,Solution,"(0, 12)","(0, 12)",0.003731,0.002100,1.988504e-14,8.270188e-11,NaN
2650,2650,Hemoglobin,NaCl 0.9%,"(0, 12)","(0, 12)",0.006434,0.001433,2.010740e-14,8.362667e-11,NaN
552,552,Platelets,Cefazolin,"(0, 12)","(0, 12)",0.009288,0.028792,3.206990e-14,1.333787e-10,NaN
